# Selenium 활용 데이터 수집 - 요일별 네이버 웹툰 목록

### 개요

- selenium: 웹 브라우저를 제어할 수 있는 파이썬 패키지
- chromedriver: selenium과 Google Chrome 브라우저를 연결하는 프로그램

## 설치해야 하는 패키지
- 파이썬 스스로 chromedriver를 내려받도록 하기 위해 `chromedriver_autoinstaller` 패키지가 필요함
```
$ pip3 install --upgrade chromedriver_autoinstaller
$ pip3 install --upgrade selenium
```

## 1. 패키지 참조

In [4]:
# ChromeDriver 자동 설치 모듈
import chromedriver_autoinstaller
# Chrome을 제어하기 위한 객체
from selenium import webdriver
# Chrome이 웹 페이지 로딩을 완료 할 때까지 최대 n초간 대기하는 기능.
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from pandas import DataFrame
# 파이썬 프로그램에 지정된 시간동안 랙을 거는 기능을 위해 사용
import time

## 2. 크롬브라우저 가동하기

In [5]:
# 크롬드라이버 자동 설치
chromedriver_autoinstaller.install()

# 크롬드라이버를 통해 크롬을 실행시킴
# -> driver 객체는 Chrome 자체
driver = webdriver.Chrome()

# 크롬브라우저가 준비될 때 까지 최대 5초씩 대기
driver.implicitly_wait(5)

CHROME >= 115, using mac-arm64 as architecture identifier


In [6]:
# 요일별 네이버 웹툰에 접근하기 위한 변수값
# -> URL을 분석하여 얻어낸 값
params = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun', 'dailyPlus']

# 네이버 웹툰의 주소 형식
naverWebtoonUrl = "https://comic.naver.com/webtoon?tab={0}"

In [7]:
# 수집된 결과가 누적될 빈 리스트
naverWebtoolData = []

# 요일별 반복
for p in params:
    # 특정 요일의 네이버 웹툰 페이지
    url = naverWebtoonUrl.format(p)
    print(url)

    # 크롬브라우저로 방문시킴
    driver.get(url)
    time.sleep(1)

    # 브라우저에 표시되는 전체 코드를 추출 --> bs4 객체로 변환
    soup = BeautifulSoup(driver.page_source)
    #print(soup)

    # 웹툰이 표시되는 부분만 추출
    webtoolList = soup.select(".ContentList__content_list--q5KXY > .item")
    #print(webtoolList)

    # 추출된 웹툰 목록 수 만큼 반복
    for w in webtoolList:
        # 포스터 URL 가져오기
        poster = w.select(".Poster__image--d9XTI")

        # 가져온 이미지가 존재하고, src속성이 있다면?
        if poster and "src" in poster[0].attrs:
            posterValue = poster[0].attrs['src']
        else:
            posterValue = None

        #print(posterValue)

        # 웹툰의 URL 가져오기
        url = w.select(".Poster__link--sopnC")

        if url and "href" in url[0].attrs:
            urlValue = url[0].attrs['href']

            if urlValue.find("https://comic.naver.com") == -1:
                urlValue = "https://comic.naver.com" + urlValue
        else:
            urlValue = None

        #print(urlValue)

        # 웹툰 제목 가져오기
        title = w.select(".ContentTitle__title--e3qXt > .text")

        if title:
            titleValue = title[0].text.strip()
        else:
            titleValue: None
        #print(titleValue)

        # 작가 이름 가져오기
        author = w.select(".ContentAuthor__author--CTAAP")

        if author:
            authorValue = author[0].text.strip()
        else:
            authorValue = None
        #print(authorValue)

        # 별점 가져오기
        rating = w.select(".Rating__star_area--dFzsb > .text")

        if rating:
            ratingValue = rating[0].text.strip()
        else:
            ratingValue = None
        #print(ratingValue)

        # 결과 병합
        resultDic = {"요일": p, "제목": titleValue, "작가": authorValue, "별점": ratingValue, "포스터": posterValue, "URL": urlValue}
        #print(resultDic)
        naverWebtoolData.append(resultDic)

df = DataFrame(naverWebtoolData)
df.to_excel("요일별_네이버_웹툰.xlsx")
df

https://comic.naver.com/webtoon?tab=mon
https://comic.naver.com/webtoon?tab=tue
https://comic.naver.com/webtoon?tab=wed
https://comic.naver.com/webtoon?tab=thu
https://comic.naver.com/webtoon?tab=fri
https://comic.naver.com/webtoon?tab=sat
https://comic.naver.com/webtoon?tab=sun
https://comic.naver.com/webtoon?tab=dailyPlus


,요일,제목,작가,별점,포스터,URL
0,mon,12시네점심,조석,9.89,https://image-comic.pstatic.net/webtoon/819217...,https://comic.naver.com/webtoon/list?titleId=8...
1,mon,뷰티풀 군바리,설이 / 윤성원,9.80,https://image-comic.pstatic.net/webtoon/648419...,https://comic.naver.com/webtoon/list?titleId=6...
2,mon,왕과의 야행,보리,9.98,https://image-comic.pstatic.net/webtoon/817859...,https://comic.naver.com/webtoon/list?titleId=8...
3,mon,백수세끼,치즈,9.85,https://image-comic.pstatic.net/webtoon/733074...,https://comic.naver.com/webtoon/list?titleId=7...
4,mon,신의 탑,SIU,9.87,https://image-comic.pstatic.net/webtoon/183559...,https://comic.naver.com/webtoon/list?titleId=1...
...,...,...,...,...,...,...
796,dailyPlus,리얼메이커,리얼메이커,9.69,None,https://comic.naver.com/webtoon/list?titleId=8...
797,dailyPlus,도와줘우주,설레임 / 한겨울,9.25,None,https://comic.naver.com/webtoon/list?titleId=7...
798,dailyPlus,용한 동거인,"졔, 윤그늘 / MACHI",9.77,None,https://comic.naver.com/webtoon/list?titleId=8...
799,dailyPlus,투 두 리스트,RYO / FLEWing,9.75,None,https://comic.naver.com/webtoon/list?titleId=7...
